In [32]:
import pandas as pd
import numpy as np
import sys
sys.path.append('..') #get root directory

Now that we know what we know which features we want to use and the format we want it to be in we can try out some different models, hoping to pick out some promising ones. The first thing that needs to be done is get the data pipeline going.

# Transformation Pipeline 
Now that the preprocessing steps are defined we can wrap all of this neatly into a Pipeline, allowing us to train and test various different models more efficiently.

In [33]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from src.features import logTransformer
from src.features import cbrtTransformer
standard_scaler = StandardScaler()
one_hot_encoder = OneHotEncoder(sparse_output = False, handle_unknown = 'ignore') # Instead of a SciPy sparse matrix, we get a NumPy array

In [34]:
preprocess = ColumnTransformer([("onehot_cities", one_hot_encoder, ['city']),("log", logTransformer(), ['sqft_lot']),('cbrt', cbrtTransformer(), ['sqft_living', 'sqft_basement', 'sqft_above'])], remainder = 'passthrough')
pipeline = Pipeline([("preprocess", preprocess),('stdscaler', standard_scaler)])

In [35]:
X_train = pd.read_csv("../data/interim/X_train.csv", index_col = 0)
X_test = pd.read_csv("../data/interim/X_test.csv", index_col = 0)
y_train = pd.read_csv("../data/interim/y_train.csv", index_col = 0)
y_test = pd.read_csv("../data/interim/y_test.csv", index_col = 0)

In [36]:
X_train_transformed_data = pipeline.fit_transform(X_train) 
X_train_transformed = pd.DataFrame(data=X_train_transformed_data, index = X_train.index, columns= pipeline.get_feature_names_out())
X_train_transformed.head()

,onehot_cities__city_Algona,onehot_cities__city_Auburn,onehot_cities__city_Beaux Arts Village,onehot_cities__city_Bellevue,onehot_cities__city_Black Diamond,onehot_cities__city_Bothell,onehot_cities__city_Burien,onehot_cities__city_Carnation,onehot_cities__city_Clyde Hill,onehot_cities__city_Covington,...,log__sqft_lot,cbrt__sqft_living,cbrt__sqft_basement,cbrt__sqft_above,remainder__bedrooms,remainder__bathrooms,remainder__floors,remainder__waterfront,remainder__view,remainder__condition
1992,-0.037088,-0.199256,-0.016577,-0.256682,-0.043895,-0.077979,-0.114372,-0.066446,-0.049786,-0.101337,...,-0.316153,0.001884,1.199345,-0.586298,0.682752,-0.519269,-0.951048,-0.084819,2.240026,-0.674857
871,-0.037088,-0.199256,-0.016577,-0.256682,-0.043895,-0.077979,-0.114372,-0.066446,-0.049786,-0.101337,...,0.200884,-0.982340,-0.784361,-0.634607,-0.438230,0.445744,0.903661,-0.084819,-0.312391,-0.674857
843,-0.037088,-0.199256,-0.016577,-0.256682,-0.043895,-0.077979,-0.114372,-0.066446,-0.049786,-0.101337,...,-1.726154,-0.667192,0.711249,-0.784264,-0.438230,0.445744,0.903661,-0.084819,-0.312391,-0.674857
2708,-0.037088,-0.199256,-0.016577,-0.256682,-0.043895,-0.077979,-0.114372,-0.066446,-0.049786,-0.101337,...,-0.077051,0.671513,-0.784361,1.063531,0.682752,0.767416,0.903661,-0.084819,-0.312391,-0.674857
380,-0.037088,-0.199256,-0.016577,-0.256682,-0.043895,-0.077979,-0.114372,-0.066446,-0.049786,-0.101337,...,-0.159552,-0.104828,-0.784361,0.266403,0.682752,-0.197598,-0.023693,-0.084819,-0.312391,2.299090


In [37]:
X_test_transformed_data = pipeline.transform(X_test)
X_test_transformed = pd.DataFrame(data = X_test_transformed_data, index = X_test.index, columns = pipeline.get_feature_names_out())

y_train_transformed = np.log(y_train) # doesn't need to go through a pipeline so no need to use logTransformer class
y_test_transformed = np.log(y_test)

In [38]:
X_train_transformed.to_csv("../data/processed/X_train_transformed.csv")
X_test_transformed.to_csv("../data/processed/X_test_transformed.csv")
y_train_transformed.to_csv("../data/processed/y_train_transformed.csv")
y_test_transformed.to_csv("../data/processed/y_test_transformed.csv")

# Model Selection

Since this is a supervised learning problem and we want to predict a house price we will use a regression model. We'll try out a few: lasso, suppor vector regression, and random forest regressor. 

In [39]:
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline


In [58]:
ls_param_grid = [{'lasso__alpha' : [1.0,1.5,2.5], 'lasso__max_iter': [100,1000,10000,] , 'lasso__tol': [0.0001, 0.001, 0.00001, 1, 10]}] 

svr_params_grid = [{'svr__kernel': ['linear'], 'svr__C': [1,20,40,60,80], 'svr__epsilon': [0.1,0.01,1.0, 0.001] },
          {'svr__kernel': ['poly'], 'svr__degree': [2,4,5], 'svr__C': [1,20,40,60,80]},
          {'svr__kernel': ['rbf', 'sigmoid'], 'svr__gamma': ['auto'], 'svr__C': [0.1,1,10,30,60,100], 'svr__epsilon': [0.1,0.01,1.0, 0.001]}]

rfr_params_grid = [{
    'randomforestregressor__n_estimators': [200, 500, 800],
    'randomforestregressor__max_depth': [10, 20, 30, None],
    'randomforestregressor__min_samples_split': [2, 5, 10],
    'randomforestregressor__min_samples_leaf': [1, 2, 4],
    'randomforestregressor__max_features': ['sqrt', 'log2', 0.5],
    'randomforestregressor__criterion' : ['squared_error', 'absolute_error']}]


In [41]:
ls_pipeline = make_pipeline(pipeline, Lasso())
ls_randomized_search = RandomizedSearchCV(ls_pipeline, ls_param_grid, scoring = 'neg_root_mean_squared_error', cv = 5, n_iter = 30, random_state = 0)
ls_randomized_search.fit(X_train, y_train_transformed)


/opt/anaconda3/lib/python3.13/site-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('pipeline',
                                              Pipeline(steps=[('preprocess',
                                                               ColumnTransformer(remainder='passthrough',
                                                                                 transformers=[('onehot_cities',
                                                                                                OneHotEncoder(handle_unknown='ignore',
                                                                                                              sparse_output=False),
                                                                                                ['city']),
                                                                                               ('log',
                                                                                                logTransformer(),
                                                                                                ['sqft_lot']),
                                                                                               ('cbrt',
                                                                                                cbrtTransformer(),
                                                                                                ['sqft_living',
                                                                                                 'sqft_basement',
                                                                                                 'sqft_above'])])),
                                                              ('stdscaler',
                                                               StandardScaler())])),
                                             ('lasso', Lasso())]),
                   n_iter=30,
                   param_distributions=[{'lasso__alpha': [1.0, 1.5, 2.5],
                                         'lasso__max_iter': [100, 1000, 10000],
                                         'lasso__tol': [0.0001, 0.001, 1e-05, 1,
                                                        10]}],
                   random_state=0, scoring='neg_root_mean_squared_error')

In [ ]:
print(ls_randomized_search.best_params_)
print(-ls_randomized_search.best_score_)
ls_randomized_search_cv_results_df = pd.DataFrame(ls_randomized_search.cv_results_)
ls_randomized_search_cv_results_df.sort_values(by = 'rank_test_score')

{'lasso__tol': 0.0001, 'lasso__max_iter': 100, 'lasso__alpha': 2.5}
0.5463295432257984


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_lasso__tol,param_lasso__max_iter,param_lasso__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.010169,0.002339,0.005875,0.001597,0.00010,100,2.5,"{'lasso__tol': 0.0001, 'lasso__max_iter': 100,...",-0.546185,-0.547515,-0.55503,-0.537503,-0.545415,-0.54633,0.005584,1
27,0.004908,0.000821,0.002231,0.000157,0.00001,1000,1.0,"{'lasso__tol': 1e-05, 'lasso__max_iter': 1000,...",-0.546185,-0.547515,-0.55503,-0.537503,-0.545415,-0.54633,0.005584,1
26,0.004549,0.000219,0.002458,0.000230,0.00010,1000,2.5,"{'lasso__tol': 0.0001, 'lasso__max_iter': 1000...",-0.546185,-0.547515,-0.55503,-0.537503,-0.545415,-0.54633,0.005584,1
25,0.004561,0.000354,0.002392,0.000290,10.00000,10000,1.0,"{'lasso__tol': 10, 'lasso__max_iter': 10000, '...",-0.546185,-0.547515,-0.55503,-0.537503,-0.545415,-0.54633,0.005584,1
24,0.005248,0.000629,0.002860,0.000257,0.00001,100,2.5,"{'lasso__tol': 1e-05, 'lasso__max_iter': 100, ...",-0.546185,-0.547515,-0.55503,-0.537503,-0.545415,-0.54633,0.005584,1
23,0.005182,0.000592,0.002582,0.000383,0.00001,100,1.5,"{'lasso__tol': 1e-05, 'lasso__max_iter': 100, ...",-0.546185,-0.547515,-0.55503,-0.537503,-0.545415,-0.54633,0.005584,1
22,0.004845,0.000478,0.002517,0.000461,0.00010,1000,1.0,"{'lasso__tol': 0.0001, 'lasso__max_iter': 1000...",-0.546185,-0.547515,-0.55503,-0.537503,-0.545415,-0.54633,0.005584,1
21,0.004276,0.000233,0.002239,0.000080,1.00000,10000,1.0,"{'lasso__tol': 1, 'lasso__max_iter': 10000, 'l...",-0.546185,-0.547515,-0.55503,-0.537503,-0.545415,-0.54633,0.005584,1
20,0.004802,0.000530,0.002582,0.000240,1.00000,1000,1.0,"{'lasso__tol': 1, 'lasso__max_iter': 1000, 'la...",-0.546185,-0.547515,-0.55503,-0.537503,-0.545415,-0.54633,0.005584,1
19,0.004611,0.000305,0.002509,0.000336,0.00010,10000,2.5,"{'lasso__tol': 0.0001, 'lasso__max_iter': 1000...",-0.546185,-0.547515,-0.55503,-0.537503,-0.545415,-0.54633,0.005584,1


In [43]:
svr_pipeline = make_pipeline(pipeline, SVR())
svr_randomized_search = RandomizedSearchCV(svr_pipeline, svr_params_grid, scoring = 'neg_root_mean_squared_error', cv = 5, n_iter = 30, random_state = 0)
y_train_flattened = np.ravel(y_train_transformed) # svr excepts a 1d array not a column vector
svr_randomized_search.fit(X_train, y_train_flattened)

/opt/anaconda3/lib/python3.13/site-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('pipeline',
                                              Pipeline(steps=[('preprocess',
                                                               ColumnTransformer(remainder='passthrough',
                                                                                 transformers=[('onehot_cities',
                                                                                                OneHotEncoder(handle_unknown='ignore',
                                                                                                              sparse_output=False),
                                                                                                ['city']),
                                                                                               ('log',
                                                                                                logTransformer(),
                                                                                                ['sqft_lot']),
                                                                                               ('cbrt',
                                                                                                cbrtTransformer(),
                                                                                                ['sqft_living',
                                                                                                 'sqft_basement',
                                                                                                 'sqft_above'])])),
                                                              ('s...
                   param_distributions=[{'svr__C': [1, 20, 40, 60, 80],
                                         'svr__epsilon': [0.1, 0.01, 1.0,
                                                          0.001],
                                         'svr__kernel': ['linear']},
                                        {'svr__C': [1, 20, 40, 60, 80],
                                         'svr__degree': [2, 4, 5],
                                         'svr__kernel': ['poly']},
                                        {'svr__C': [0.1, 1, 10, 30, 60, 100],
                                         'svr__epsilon': [0.1, 0.01, 1.0,
                                                          0.001],
                                         'svr__gamma': ['auto'],
                                         'svr__kernel': ['rbf', 'sigmoid']}],
                   random_state=0, scoring='neg_root_mean_squared_error')

In [44]:
print(svr_randomized_search.best_params_)
print(-svr_randomized_search.best_score_)
svr_randomized_search_cv_results = pd.DataFrame(svr_randomized_search.cv_results_)
svr_randomized_search_cv_results.sort_values(by = 'rank_test_score')

{'svr__kernel': 'linear', 'svr__epsilon': 0.1, 'svr__C': 80}
0.2902156356612463


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svr__kernel,param_svr__degree,param_svr__C,param_svr__gamma,param_svr__epsilon,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
14,45.656957,1.892284,0.033948,0.001976,linear,NaN,80.0,NaN,0.100,"{'svr__kernel': 'linear', 'svr__epsilon': 0.1,...",-0.277722,-0.271391,-0.309711,-0.267428,-0.324827,-0.290216,0.022837,1
9,24.526478,1.458337,0.039362,0.005828,linear,NaN,40.0,NaN,0.100,"{'svr__kernel': 'linear', 'svr__epsilon': 0.1,...",-0.277695,-0.271470,-0.309672,-0.267457,-0.324861,-0.290231,0.022825,2
11,53.187856,2.004856,0.052353,0.002399,linear,NaN,60.0,NaN,0.010,"{'svr__kernel': 'linear', 'svr__epsilon': 0.01...",-0.279339,-0.272174,-0.309507,-0.267725,-0.327202,-0.291189,0.023173,3
28,1.422051,0.063487,0.053283,0.000609,linear,NaN,1.0,NaN,0.001,"{'svr__kernel': 'linear', 'svr__epsilon': 0.00...",-0.279410,-0.272031,-0.309861,-0.267822,-0.327687,-0.291362,0.023377,4
20,0.205883,0.005219,0.093216,0.003001,rbf,NaN,1.0,auto,0.100,"{'svr__kernel': 'rbf', 'svr__gamma': 'auto', '...",-0.277892,-0.271050,-0.314083,-0.282929,-0.334911,-0.296173,0.024340,5
16,0.332928,0.016085,0.153044,0.002948,rbf,NaN,1.0,auto,0.010,"{'svr__kernel': 'rbf', 'svr__gamma': 'auto', '...",-0.279012,-0.272658,-0.314138,-0.283148,-0.336478,-0.297087,0.024332,6
2,0.334541,0.002263,0.068930,0.000364,sigmoid,NaN,0.1,auto,0.001,"{'svr__kernel': 'sigmoid', 'svr__gamma': 'auto...",-0.287229,-0.283469,-0.309475,-0.278888,-0.331944,-0.298201,0.019880,7
5,0.329163,0.003319,0.067349,0.001056,sigmoid,NaN,0.1,auto,0.010,"{'svr__kernel': 'sigmoid', 'svr__gamma': 'auto...",-0.288536,-0.285866,-0.309510,-0.276234,-0.331817,-0.298392,0.019929,8
1,0.484251,0.005643,0.151266,0.004428,rbf,NaN,10.0,auto,0.010,"{'svr__kernel': 'rbf', 'svr__gamma': 'auto', '...",-0.283378,-0.292243,-0.324707,-0.296275,-0.340283,-0.307377,0.021501,9
17,0.532451,0.012705,0.165761,0.002383,rbf,NaN,10.0,auto,0.001,"{'svr__kernel': 'rbf', 'svr__gamma': 'auto', '...",-0.284092,-0.293043,-0.325400,-0.296545,-0.341254,-0.308067,0.021611,10


In [60]:
rfr_pipeline = make_pipeline(pipeline, RandomForestRegressor())
rfr_randomized_search = RandomizedSearchCV(rfr_pipeline, rfr_params_grid, scoring = 'neg_root_mean_squared_error', cv = 5, n_iter = 30, random_state = 0)
y_train_flattened = np.ravel(y_train_transformed)
rfr_randomized_search.fit(X_train, y_train_flattened)



/opt/anaconda3/lib/python3.13/site-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('pipeline',
                                              Pipeline(steps=[('preprocess',
                                                               ColumnTransformer(remainder='passthrough',
                                                                                 transformers=[('onehot_cities',
                                                                                                OneHotEncoder(handle_unknown='ignore',
                                                                                                              sparse_output=False),
                                                                                                ['city']),
                                                                                               ('log',
                                                                                                logTransformer(),
                                                                                                ['sqft_lot']),
                                                                                               ('cbrt',
                                                                                                cbrtTransformer(),
                                                                                                ['sqft_living',
                                                                                                 'sqft_basement',
                                                                                                 'sqft_above'])])),
                                                              ('s...
                   param_distributions=[{'randomforestregressor__criterion': ['squared_error',
                                                                              'absolute_error'],
                                         'randomforestregressor__max_depth': [10,
                                                                              20,
                                                                              30,
                                                                              None],
                                         'randomforestregressor__max_features': ['sqrt',
                                                                                 'log2',
                                                                                 0.5],
                                         'randomforestregressor__min_samples_leaf': [1,
                                                                                     2,
                                                                                     4],
                                         'randomforestregressor__min_samples_split': [2,
                                                                                      5,
                                                                                      10],
                                         'randomforestregressor__n_estimators': [200,
                                                                                 500,
                                                                                 800]}],
                   random_state=0, scoring='neg_root_mean_squared_error')

In [61]:
print(-rfr_randomized_search.best_score_)
print(rfr_randomized_search.best_params_)
rfr_randomized_search_cv_results = pd.DataFrame(rfr_randomized_search.cv_results_)
rfr_randomized_search_cv_results.sort_values(by = 'rank_test_score')

0.3000976156983409
{'randomforestregressor__n_estimators': 800, 'randomforestregressor__min_samples_split': 10, 'randomforestregressor__min_samples_leaf': 1, 'randomforestregressor__max_features': 'log2', 'randomforestregressor__max_depth': None, 'randomforestregressor__criterion': 'absolute_error'}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_randomforestregressor__n_estimators,param_randomforestregressor__min_samples_split,param_randomforestregressor__min_samples_leaf,param_randomforestregressor__max_features,param_randomforestregressor__max_depth,param_randomforestregressor__criterion,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
17,33.960247,0.461970,0.071164,0.001110,800,10,1,log2,None,absolute_error,"{'randomforestregressor__n_estimators': 800, '...",-0.283544,-0.285677,-0.313602,-0.285413,-0.332253,-0.300098,0.019565,1
1,23.774798,0.286459,0.044472,0.000724,500,10,1,sqrt,30,absolute_error,"{'randomforestregressor__n_estimators': 500, '...",-0.283474,-0.289016,-0.313629,-0.286273,-0.333844,-0.301247,0.019522,2
28,1.125619,0.003662,0.062415,0.002247,500,2,1,log2,None,squared_error,"{'randomforestregressor__n_estimators': 500, '...",-0.286692,-0.285726,-0.312276,-0.290372,-0.331290,-0.301271,0.017862,3
21,1.440507,0.022280,0.071268,0.004060,800,10,1,sqrt,None,squared_error,"{'randomforestregressor__n_estimators': 800, '...",-0.286427,-0.288468,-0.313450,-0.290953,-0.333778,-0.302615,0.018367,4
12,0.457174,0.030130,0.024012,0.002359,200,5,1,sqrt,30,squared_error,"{'randomforestregressor__n_estimators': 200, '...",-0.285994,-0.289524,-0.313226,-0.292338,-0.332784,-0.302773,0.017753,5
19,23.609772,0.112495,0.044968,0.001377,500,5,1,sqrt,20,absolute_error,"{'randomforestregressor__n_estimators': 500, '...",-0.286137,-0.290088,-0.315019,-0.288990,-0.335081,-0.303063,0.019084,6
10,85.909040,1.561475,0.066133,0.005250,800,10,2,0.5,20,absolute_error,"{'randomforestregressor__n_estimators': 800, '...",-0.287002,-0.291385,-0.313240,-0.290560,-0.333418,-0.303121,0.017752,7
15,0.381419,0.019036,0.020906,0.002757,200,10,1,sqrt,None,squared_error,"{'randomforestregressor__n_estimators': 200, '...",-0.287161,-0.288731,-0.314262,-0.292305,-0.333639,-0.303220,0.018079,8
14,24.364936,0.531284,0.054365,0.006896,500,2,1,sqrt,20,absolute_error,"{'randomforestregressor__n_estimators': 500, '...",-0.287093,-0.289244,-0.314531,-0.291758,-0.335294,-0.303584,0.018670,9
3,24.120286,0.207845,0.021710,0.000302,200,5,1,0.5,None,absolute_error,"{'randomforestregressor__n_estimators': 200, '...",-0.284043,-0.286644,-0.314203,-0.295532,-0.338393,-0.303763,0.020287,10


After running the randomized search we can an idea of which hyperparmeters we should have set and which ones we can still play around with. For Lasso it's rather hard to tell what hyperparameters might be best to go with since they all ranked 1st in the cv results. However svr's and rfr's scoring are closer to 0 by a considerdable margin since our scoring is in log-space and there is a better sense of what hyperparameters to go with. Running an exhaustive grid search on the promising hyperparameters can hopefully squeeze out some better scoring and translate into better predictions.

In [68]:
from sklearn.model_selection import GridSearchCV

svr_grid_search_params = [{
    'svr__kernel': ['linear'], 
    'svr__epsilon': [0.1,0.01,1.0,0.001],
    'svr__C': [40,60,80,100,120] }]

rfr_grid_search_params = [{
    'randomforestregressor__n_estimators': [650,800,950],
    'randomforestregressor__max_depth': [10, None],
    'randomforestregressor__min_samples_split': [10, 15, 20],
    'randomforestregressor__min_samples_leaf': [1],
    'randomforestregressor__max_features': ['sqrt', 'log2'],
    'randomforestregressor__criterion' : ['absolute_error']}]


In [48]:
svr_grid_search = GridSearchCV(svr_pipeline, svr_grid_search_params, scoring = 'neg_root_mean_squared_error', cv = 5)
svr_grid_search.fit(X_train, y_train_flattened)

/opt/anaconda3/lib/python3.13/site-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('pipeline',
                                        Pipeline(steps=[('preprocess',
                                                         ColumnTransformer(remainder='passthrough',
                                                                           transformers=[('onehot_cities',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False),
                                                                                          ['city']),
                                                                                         ('log',
                                                                                          logTransformer(),
                                                                                          ['sqft_lot']),
                                                                                         ('cbrt',
                                                                                          cbrtTransformer(),
                                                                                          ['sqft_living',
                                                                                           'sqft_basement',
                                                                                           'sqft_above'])])),
                                                        ('stdscaler',
                                                         StandardScaler())])),
                                       ('svr', SVR())]),
             param_grid=[{'svr__C': [40, 60, 80, 100, 120],
                          'svr__epsilon': [0.1, 0.01, 1.0, 0.001],
                          'svr__kernel': ['linear']}],
             scoring='neg_root_mean_squared_error')

In [65]:
print(-svr_grid_search.best_score_)
print(svr_grid_search.best_params_)

0.29019480244638035
{'svr__C': 100, 'svr__epsilon': 0.1, 'svr__kernel': 'linear'}


In [69]:
rfr_grid_search = GridSearchCV(rfr_pipeline, rfr_grid_search_params, scoring = 'neg_root_mean_squared_error', cv = 5)
rfr_grid_search.fit(X_train, y_train_flattened)

/opt/anaconda3/lib/python3.13/site-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('pipeline',
                                        Pipeline(steps=[('preprocess',
                                                         ColumnTransformer(remainder='passthrough',
                                                                           transformers=[('onehot_cities',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False),
                                                                                          ['city']),
                                                                                         ('log',
                                                                                          logTransformer(),
                                                                                          ['sqft_lot']),
                                                                                         ('cbrt',
                                                                                          cbrtTransformer(),
                                                                                          ['sqft_living',
                                                                                           'sqft_basement',
                                                                                           'sqft_above'])])),
                                                        ('stdscal...
             param_grid=[{'randomforestregressor__criterion': ['absolute_error'],
                          'randomforestregressor__max_depth': [10, None],
                          'randomforestregressor__max_features': ['sqrt',
                                                                  'log2'],
                          'randomforestregressor__min_samples_leaf': [1],
                          'randomforestregressor__min_samples_split': [10, 15,
                                                                       20],
                          'randomforestregressor__n_estimators': [650, 800,
                                                                  950]}],
             scoring='neg_root_mean_squared_error')

In [70]:
print(-rfr_grid_search.best_score_)
print(rfr_grid_search.best_estimator_)

0.30007231290261505
Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('preprocess',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('onehot_cities',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False),
                                                                   ['city']),
                                                                  ('log',
                                                                   logTransformer(),
                                                                   ['sqft_lot']),
                                                                  ('cbrt',
                                                                   cbrtTransformer(),
                                                     

The Joblib library will be used to store the results and fitted models to avoid having to rerun the code cells, helping to save time.

In [71]:
import joblib
import json 

In [74]:
with open("../models/svr_grid_model_score.json", "w") as f:
    json.dump(svr_grid_search.best_score_, f, indent=4)
    
with open("../models/rfr_grid_model_score.json", "w") as f:
    json.dump(rfr_grid_search.best_score_, f, indent=4)

In [73]:
joblib.dump(svr_grid_search.best_estimator_, '../models/svr_model.joblib')
joblib.dump(rfr_grid_search.best_estimator_, '../models/rfr_model.joblib')

['../models/rfr_model.joblib']